# Task 3: Build and evaluate Q&A Application using Amazon Bedrock Knowledge Bases using RAG Assessment (RAGAS) framework

In this task, you build and evaluate a Q&A application using LangChain's *AmazonKnowledgeBasesRetriever* class, Chains, and the RAGAS framework for evaluating the responses. Here, you query the knowledge base to get the desired number of document chunks based on similarity search. Next, you prompt the text generation LLM by supplying the document chunks as context along with the query. Then you evaluate the responses using the following evaluation metrics: faithfulness, answer_relevancy, context_recall, context_precision, context_entity_recall, answer_similarity, answer_correctness, harmfulness, maliciousness, coherence, correctness and conciseness.

<i aria-hidden="true" class="fas fa-info-circle" style="color:#007FAA"></i> **Learn more:** Refer to *[Metrics](https://docs.ragas.io/en/latest/concepts/metrics/index.html)* for additional information about different metrics used with the Ragas framework.

### Pattern

You implement the solution using Retrieval Augmented Generation (RAG) pattern. RAG retrieves data from outside the language model and augments the prompts by adding the relevant retrieved data in context. In this task, you create responses to queries using the knowledge base that was already created during lab provisioning.

#### Evaluation

- You utilize RAGAS for evaluation on the following metrics:
  - **Faithfulness:** This measures the factual consistency of the generated answer against the given context. It is calculated from answer and retrieved context. The answer is scaled to (0,1) range. Higher the better.
  - **Answer Relevance:** This metric focuses on assessing how pertinent the generated answer is to the given prompt. A lower score is assigned to answers that are incomplete or contain redundant information and higher scores indicate better relevancy. This metric is computed using the question, the context and the answer. Please note, that even though in practice the score will range between 0 and 1 most of the time, this is not mathematically guaranteed, due to the nature of the cosine similarity ranging from -1 to 1.
  - **Context Precision:** This is a metric that evaluates whether all of the ground-truth relevant items present in the contexts are ranked higher or not. Ideally all the relevant chunks must appear at the top ranks. This metric is computed using the question, ground_truth and the contexts, with values ranging between 0 and 1, where higher scores indicate better precision.
  - **Context Recall:** This metric measures the extent to which the retrieved context aligns with the annotated answer, treated as the ground truth. It is computed based on the ground truth and the retrieved context, and the values range between 0 and 1, with higher values indicating better performance.
  - **Context entities recall:** This metric gives the measure of recall of the retrieved context, based on the number of entities present in both ground_truths and contexts relative to the number of entities present in the ground_truths alone. Simply put, it is a measure of what fraction of entities are recalled from ground_truths. This metric is useful in fact-based use cases like tourism help desk, historical QA, etc. This metric can help evaluate the retrieval mechanism for entities, based on comparison with entities present in ground_truths, because in cases where entities matter, we need the contexts which cover them.
  - **Answer Semantic Similarity:** The concept of Answer Semantic Similarity pertains to the assessment of the semantic resemblance between the generated answer and the ground truth. This evaluation is based on the ground truth and the answer, with values falling within the range of 0 to 1. A higher score signifies a better alignment between the generated answer and the ground truth.
  - **Answer Correctness:** The assessment of Answer Correctness involves gauging the accuracy of the generated answer when compared to the ground truth. This evaluation relies on the ground truth and the answer, with scores ranging from 0 to 1. A higher score indicates a closer alignment between the generated answer and the ground truth, signifying better correctness. Answer correctness encompasses two critical aspects: semantic similarity between the generated answer and the ground truth, as well as factual similarity. These aspects are combined using a weighted scheme to formulate the answer correctness score. Users also have the option to employ a ‘threshold’ value to round the resulting score to binary, if desired.
  - **Aspect Critique:** This is designed to assess submissions based on predefined aspects such as harmlessness and correctness. The output of aspect critiques is binary, indicating whether the submission aligns with the defined aspect or not. This evaluation is performed using the ‘answer’ as input.

In this task, you use AnyCompany's financial 10k reports (synthetically generated dataset) as a text corpus to perform Q&A on. This data is already ingested into the Knowledge Base in Amazon Bedrock.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** In your specific use case, you can sync different files for different domain topics and query this notebook in the same manner to evaluate model responses using the retrieve API from knowledge bases.

<i aria-hidden="true" class="fas fa-exclamation-circle" style="color:#7C5AED"></i> **Caution:** It is recommended to run each code cell individually rather than using the **Run All Cells** option from the **Run** menu. Running all cells together can sometimes lead to unexpected behavior, such as the Kernel crashing or restarting. By executing cells one by one, you can better control the execution flow, catch potential errors early, and ensure that your code runs as intended.

## Task 3.1: Setup the environment

To run this notebook you need to install dependencies, LangChain and RAGAS and updated boto3, botocore packages.

You set up the necessary packages by following the steps provided below:

- Import the necessary libraries for creating *bedrock-runtime* for invoking foundation models
- Import LangChain related libraries
- Initialize bedrock model **amazon.titan-text-premier-v1:0** as your large language model to perform query completions using the RAG pattern.
- Initialize bedrock model **amazon.nova-lite-v1:0** as your large language model to perform RAG evaluation.
- Initialize bedrock model **amazon.titan-embed-text-v2:0** as your large language embedding model to create embeddings for RAG evaluation. This is the same embedding model that was used to create the knowledge base.
- Initialize LangChain retriever integrated with knowledge bases.
- Later in the notebook you wrap the LLM and retriever as a chain for building your Q&A application.

1. Run the following code cell to verify the ID for the existing Knowledge Base in Amazon Bedrock:

In [1]:
import botocore
import boto3

session = boto3.Session()
bedrock_client = session.client('bedrock-agent')

try:
    response = bedrock_client.list_knowledge_bases(
        maxResults=1  # We only need to retrieve the first Knowledge Base
    )
    knowledge_base_summaries = response.get('knowledgeBaseSummaries', [])

    if knowledge_base_summaries:
        kb_id = knowledge_base_summaries[0]['knowledgeBaseId']
        print(f"Knowledge Base ID: {kb_id}")
    else:
        print("No Knowledge Base summaries found.")
        
except botocore.exceptions.ClientError as e:
    print(f"Error: {e}")

Knowledge Base ID: W1FOJLAZNW


2. Run the following code cell to install dependencies:

In [2]:
import boto3
import pprint

from langchain_aws import ChatBedrock
from langchain_aws import BedrockEmbeddings
from langchain_aws.retrievers.bedrock import AmazonKnowledgeBasesRetriever


pp = pprint.PrettyPrinter(indent=2)

bedrock_client = boto3.client('bedrock-runtime')

## 두 개의 에이전트를 사용
llm_for_text_generation = ChatBedrock(model_id="amazon.nova-lite-v1:0", client=bedrock_client)
llm_for_evaluation = ChatBedrock(model_id="amazon.nova-lite-v1:0", client=bedrock_client)

bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v2:0",client=bedrock_client)

## Task 3.2: Create *AmazonKnowledgeBasesRetriever* object from LangChain

In this task, you create a *AmazonKnowledgeBasesRetriever* object from LangChain to search the knowledge base and return the relevant results, giving you more control to build custom workflows on top of the semantic search results.

3. Run the following code cell to create a *AmazonKnowledgeBasesRetriever* object:

In [3]:
retriever = AmazonKnowledgeBasesRetriever(
        knowledge_base_id=kb_id,
        retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 5}},
        # endpoint_url=endpoint_url,
        # region_name="us-east-1",
        # credentials_profile_name="<profile_name>",
    )

## Task 3.3: Model Invocation and Response Generation using RetrievalQA chain 

In this task, you invoke the model and visualize the response using the following information:

Question =

```
Provide a list of few risks for AnyCompany financial in numbered list without description."
```

Ground truth answer = 

```
1. Commodity Prices
2. Foreign Exchange Rates 
3. Equity Prices
4. Credit Risk
5. Liquidity Risk
...
...
```

4. Run the following code cell to create a prompt with context and question as variables:

In [4]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE = """
Human: You are a financial advisor AI system, and provides answers to questions by using fact based and statistical information when possible. 
Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
<context>
{context}
</context>

<question>
{question}
</question>

The response should be specific and use statistics or numbers when possible.

Assistant:"""
prompt = PromptTemplate(template=PROMPT_TEMPLATE, 
                               input_variables=["context","question"])

5. Run the following code cell to invoke the model using a pre-defined query and print the results:

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs): #concatenate the text from the page_content field in the output from retriever.invoke
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm_for_text_generation
    | StrOutputParser()
)

query = "Provide a list of ten risks for AnyCompany financial as a numbered list. Do not include descriptions."

response=chain.invoke(query)
print(response)

1. Commodity Prices
2. Foreign Exchange Rates
3. Equity Prices
4. Operational Risk
5. Regulatory Risk
6. Strategic Risk
7. Reputation Risk
8. Legal Risk
9. Environmental, Social, and Governance (ESG) Risk
10. Liquidity Risk


## Task 3.4: Prepare the evaluation data

As RAGAS aims to be a reference-free evaluation framework, the required preparations of the evaluation dataset are minimal. In this task, you prepare the *question* and *ground_truths* pairs from which you can prepare the remaining information through inference as shown below.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** If you are not interested in the *context_recall* metric, you don’t need to provide the *ground_truths* information. In this task, all you need to prepare are the *questions*.

5. Run the following code cell to prepare the *question* and *ground_truths* pairs for evaluation. This may run for a few minutes with retries in case of throttling:

In [6]:
from datasets import Dataset
import time
import random

# Define questions and ground truths for RAGAS evaluation
questions = [
    "What was the primary reason for the increase in net cash by operating activities for AnyCompany Financial in 2021?",
    "Which year did AnyCompany Financial have the highest net cash used in investing activities, what was the primary reason?",
    "What was the primary source of cash inflows from financing activities for AnyCompany Financial in 2021?",
    "Calculate the year-over-year percentage change in cash and cash equivalents for AnyCompany Financial from 2020 to 2021.",
    "With the information provided, what can you infer about AnyCompany Financial's overall financial health and growth prospects?"
]

ground_truth = [
    "An increase in net cash provided by operating activities was primarily due to increases in net income and favorable changes in operating assets and liabilities.",
    "AnyCompany Financial had the highest net cash used in investing activities in 2021, at $360 million, compared to $290 million in 2020 and $240 million in 2019. The primary reason was an increase in purchases of property, plant, and equipment and marketable securities.",
    "The primary source of cash inflows from financing activities for AnyCompany Financial in 2021 was an increase in proceeds from the issuance of common stock and long-term debt.",
    "To calculate the year-over-year percentage change in cash and cash equivalents from 2020 to 2021: \
    2020 cash and cash equivalents: $350 million \
    2021 cash and cash equivalents: $480 million \
    Percentage change = (2021 value - 2020 value) / 2020 value * 100 \
    = ($480 million - $350 million) / $350 million * 100 \
    = 37.14% increase",
    "Based on information provided, AnyCompany Financial appears to be in a healthy financial position with good growth prospects. The company increased its net cash provided by operating activities, indicating strong profitability and efficient management of working capital. AnyCompany Financial has been investing in long-term assets, such as property, plant, and equipment, and marketable securities, which suggests plans for future growth and expansion. The company was able to finance its growth through the issuance of common stock and long-term debt, indicating confidence from investors and lenders. Overall, AnyCompany Financial's steady increase in cash and cash equivalents over the past three years provides a strong foundation for future growth and investment opportunities."
]

def get_model_response(query, chain, retriever, max_retries=5, wait_time=15):
    """Get response from the model with fixed wait time between retries"""
    for attempt in range(max_retries):
        try:
            # Configure Nova Lite with increased tokens
            nova_config = {
                "schemaVersion": "messages-v1",
                "messages": [{
                    "role": "user",
                    "content": [{"text": query}]
                }],
                "inferenceConfig": {
                    "maxTokens": 2048,
                    "temperature": 0.5,
                    "topP": 0.9,
                    "topK": 20
                }
            }
            
            # Try to invoke with config override
            try:
                answer = chain.invoke(
                    query,
                    config_override={"model_kwargs": nova_config}
                )
            except AttributeError:
                # If config_override doesn't work, try direct invocation
                answer = chain.invoke(query)
            
            context = [docs.page_content for docs in retriever.invoke(query)]
            print(f"Successfully processed query on attempt {attempt + 1}")
            return answer, context
            
        except Exception as e:
            if attempt == max_retries - 1:
                print(f"Failed after {max_retries} attempts for query: {query[:50]}...")
                print(f"Error: {str(e)}")
                return None, None
            print(f"Attempt {attempt + 1} failed, waiting {wait_time} seconds before retry...")
            time.sleep(wait_time)

# Process questions one at a time with fixed delay
answers = []
contexts = []

print("Starting to process questions...")
for i, query in enumerate(questions, 1):
    print(f"\nProcessing question {i}/{len(questions)}")
    print(f"Query: {query[:100]}...")
    
    answer, context = get_model_response(query, chain, retriever)
    if answer is not None:
        answers.append(answer)
        contexts.append(context)
        print(f"Successfully processed question {i}")
    else:
        print(f"Failed to process question {i}")
    
    #if i < len(questions):
    #    print(f"Waiting 60 seconds before next question...")
    #    time.sleep(60)

# Create dataset for RAGAS evaluation
data = {
    "question": questions[:len(answers)],
    "ground_truth": ground_truth[:len(answers)],
    "answer": answers,
    "contexts": contexts
}

# Convert to dataset
dataset = Dataset.from_dict(data)

# Print dataset information
print("\nDataset Creation Summary:")
print(f"Total questions processed: {len(dataset)} out of {len(questions)}")
print(f"Columns available: {dataset.column_names}")

# Print sample entry
if len(dataset) > 0:
    print("\nSample Entry (First Question):")
    print(f"Question: {dataset[0]['question']}")
    print(f"Ground Truth: {dataset[0]['ground_truth']}")
    print(f"Model Answer: {dataset[0]['answer']}")
else:
    print("\nNo entries were successfully processed into the dataset.")


Starting to process questions...

Processing question 1/5
Query: What was the primary reason for the increase in net cash by operating activities for AnyCompany Fina...
Successfully processed query on attempt 1
Successfully processed question 1

Processing question 2/5
Query: Which year did AnyCompany Financial have the highest net cash used in investing activities, what was...
Successfully processed query on attempt 1
Successfully processed question 2

Processing question 3/5
Query: What was the primary source of cash inflows from financing activities for AnyCompany Financial in 20...
Successfully processed query on attempt 1
Successfully processed question 3

Processing question 4/5
Query: Calculate the year-over-year percentage change in cash and cash equivalents for AnyCompany Financial...
Successfully processed query on attempt 1
Successfully processed question 4

Processing question 5/5
Query: With the information provided, what can you infer about AnyCompany Financial's overall 

6. Run the following code cell to see the answers from the LLM and the ground truths for the evaluation set of questions:

In [7]:
i=0
for answer in answers:
    i=i+1
    print(str(i)+').'+questions[i-1]+'\n')
    print("LLM:" +answer+'\n')
    print ("Ground truth: "+ ground_truth[i-1]+'\n')

1).What was the primary reason for the increase in net cash by operating activities for AnyCompany Financial in 2021?

LLM:The primary reason for the increase in net cash provided by operating activities for AnyCompany Financial in 2021 was an increase in net income and favorable changes in operating assets and liabilities. Specifically, net cash provided by operating activities increased from $880 million in 2020 to $1,100 million in 2021. This growth was driven by the increase in net income, which rose from $1,125 million in 2020 to $1,200 million in 2021, and favorable changes in operating assets and liabilities.

Ground truth: An increase in net cash provided by operating activities was primarily due to increases in net income and favorable changes in operating assets and liabilities.

2).Which year did AnyCompany Financial have the highest net cash used in investing activities, what was the primary reason?

LLM:AnyCompany Financial had the highest net cash used in investing activi

## Task 3.5: Evaluate the RAG application

In this task, you import all the metrics you want to use from *ragas.metrics*. Then, you use the *evaluate()* function and simply pass in the relevant metrics and the prepared dataset.

7. Run the following code cell to import all the metrics from *ragas.metrics* and use the *evaluate()* function:

In [ ]:
import warnings
import logging

warnings.filterwarnings('ignore')   # ignore warnings related to pydantic v1 to v2 migration
logging.getLogger('root').setLevel(logging.CRITICAL)

from datasets import Dataset
if not hasattr(Dataset, 'from_list'):
    def from_list_compatibility(data_list):
        if isinstance(data_list, list) and len(data_list) > 0 and isinstance(data_list[0], dict):
            keys = data_list[0].keys()
            data_dict = {key: [item[key] for item in data_list] for key in keys}
            return Dataset.from_dict(data_dict)
        return Dataset.from_dict({})
    Dataset.from_list = staticmethod(from_list_compatibility)

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    context_entity_recall,
    answer_similarity,
    answer_correctness
)

from ragas.metrics.critique import (
harmfulness, 
maliciousness, 
coherence, 
correctness, 
conciseness
)

#specify the metrics here
metrics = [
        faithfulness,
        answer_relevancy,
        context_precision,
        context_recall,
        context_entity_recall,
        answer_similarity,
        answer_correctness,
        harmfulness, 
        maliciousness, 
        coherence, 
        correctness, 
        conciseness
    ]

try:
    result = evaluate(
        dataset=dataset,
        metrics=metrics,
        llm=llm_for_evaluation,
        embeddings=bedrock_embeddings,
    )
    df = result.to_pandas()
except Exception as e:
    # Handle any exceptions that occur during the evaluation
    print(f"An error occurred: {e}")

<i aria-hidden="true" class="fas fa-exclamation-circle" style="color:#7C5AED"></i> **Caution:** You can safely ignore the warnings in the above output. Make sure that evaluation is completed 100% before proceeding further. This step takes approximately 7 to 10 minutes to complete.

8. Run the below code cell to see the resulting RAGAS scores:

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 10
df.style.set_sticky(axis="columns")

9. Run the below code cell to export the resulting RAGAS scores in excel format:

In [7]:
df.style.to_excel('styled.xlsx', engine='openpyxl')

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** After successfully running the above code cell, a file named *styled.xlsx* should be available for review in the left navigation pane under the **en_us** folder. If the file takes too long to open, right click the file and choose **Open in New Browser Tab**.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Note:** Please note that the scores above gives a relative idea on the performance of your RAG application and should be used with caution and not as standalone scores. Also note, that you have used only 5 question/answer pairs for evaluation. As a best practice, you should use enough data to cover different aspects of your document for evaluating model.

Based on the scores, you can review other components of your RAG workflow to further optimize the scores. Few recommended options are to review your chunking strategy, prompt instructions, adding more numberOfResults for additional context and so on. 

<i aria-hidden="true" class="far fa-thumbs-up" style="color:#008296"></i> **Task complete:** You have completed this notebook. To move to the next part of the lab, do the following:

- Close this notebook file.
- Return to the lab session and continue with Task 4.